# 0. Introduction
1. We have two `class` to extract the information neighbors from center atoms, they are all derived class of `StructureNeighborsBase`
    1. `matersdk.io.publicLayer.neigh.StructureNeighborsV1`: Use `NearestNeighbors` in `sklearn`
    2. `matersdk.io.publicLayer.neigh.StructureNeighborsV2`: Implemet custom `NearestNeighbors`
    3. Now `StructureNeighborsV2` is much faster than `StructureNeighborsV1`

In [1]:
from timeit import default_timer as timer
from matersdk.io.publicLayer.structure import DStructure
from matersdk.io.publicLayer.neigh import StructureNeighborsDescriptor

# 1. Demo for `StructureNeighborsV1`
1. Running time: 1.66 s

In [2]:
atom_config_path = "/data/home/liuhanyu/hyliu/code/matersdk/demo/structure/atom.config"
scaling_matrix = [5, 5, 1]  # Expand the primitive cell to process pbc
reformat_mark = True    # Resort the atoms according to atomic number after expanding to supercell
n_neighbors = 200   # The total number of all kinds of atoms. You'd better set it larger than `max_num_nbr`(usually 100) in `DeepPot-SE`
algorithm = "ball_tree" # The algorithm for `sklearn.NearestNerghbors`
coords_are_cartesian = True # Use cartesian coordinates to find neighbors

_start = timer()
structure = DStructure.from_file(file_format="pwmat", file_path=atom_config_path)
struct_neighbors = StructureNeighborsDescriptor.create(
                    'v1',
                    structure=structure,
                    scaling_matrix=scaling_matrix,
                    reformat_mark=reformat_mark,
                    coords_are_cartesian=coords_are_cartesian,
                    n_neighbors=n_neighbors,
                    algorithm=algorithm)
_end = timer()

print("\t1.1. The number of atoms in primitive cell:\t", len(struct_neighbors.structure.species))
print("\t1.2. The shape of key_nbr_atomic_numbers:\t", struct_neighbors.key_nbr_atomic_numbers.shape)
print("\t1.3. The shape of key_nbr_distances:\t", struct_neighbors.key_nbr_distances.shape)
print("\t1.4. The shape of key_nbr_coords:\t", struct_neighbors.key_nbr_coords.shape)
print("Running time: {0} s".format(_end-_start))

	1.1. The number of atoms in primitive cell:	 12
	1.2. The shape of key_nbr_atomic_numbers:	 (12, 200)
	1.3. The shape of key_nbr_distances:	 (12, 200)
	1.4. The shape of key_nbr_coords:	 (12, 200, 3)
Running time: 1.6094769388437271 s


# 2. Demo for `StructureNeighborsV2`
1. Running time: 0.037 s

In [3]:
atom_config_path = "/data/home/liuhanyu/hyliu/code/matersdk/demo/structure/atom.config"
scaling_matrix = [5, 5, 1]  # Expand the primitive cell to process pbd
reformat_mark = True    # Resort the atoms according to atomic number after expanding to supercell
n_neighbors = 200   # The total number of all kinds of atoms. You'd better set it larger than `max_num_nbr`(usually 100) in `DeepPot-SE`
coords_are_cartesian = True # Use cartesian coordinates to find neighbors

_start = timer()
structure = DStructure.from_file(file_format="pwmat", file_path=atom_config_path)
struct_neighbors = StructureNeighborsDescriptor.create(
                    'v2',
                    structure=structure,
                    scaling_matrix=scaling_matrix,
                    reformat_mark=reformat_mark,
                    coords_are_cartesian=coords_are_cartesian,
                    n_neighbors=n_neighbors)
_end = timer()

print("\t1.1. The number of atoms in primitive cell:\t", len(struct_neighbors.structure.species))
print("\t1.2. The shape of key_nbr_atomic_numbers:\t", struct_neighbors.key_nbr_atomic_numbers.shape)
print("\t1.3. The shape of key_nbr_distances:\t", struct_neighbors.key_nbr_distances.shape)
print("\t1.4. The shape of key_nbr_coords:\t", struct_neighbors.key_nbr_coords.shape)
print("Running time: {0} s".format(_end-_start))

	1.1. The number of atoms in primitive cell:	 12
	1.2. The shape of key_nbr_atomic_numbers:	 (12, 200)
	1.3. The shape of key_nbr_distances:	 (12, 200)
	1.4. The shape of key_nbr_coords:	 (12, 200, 3)
Running time: 0.034351348876953125 s


# 3. Demo for `StructureNeighborsV3`
1. Running time: 0.007814742624759674 s

In [7]:
atom_config_path = "/data/home/liuhanyu/hyliu/code/matersdk/demo/structure/atom.config"
scaling_matrix = [3, 3, 1]
reformat_mark = True
rcut = 3.2
coords_are_cartesian = True   


_start = timer()
structure = DStructure.from_file(file_format="pwmat", file_path=atom_config_path)
neighbors_v3 = StructureNeighborsDescriptor.create(
                    'v3',
                    structure=structure,
                    scaling_matrix=scaling_matrix,
                    reformat_mark=reformat_mark,
                    coords_are_cartesian=coords_are_cartesian,
                    rcut=rcut)
_end = timer()


### Step 1.
print()
print("Step 1. 在截断半径 {0} 内，最大近邻原子数为:".format(rcut), end="\t")
print(neighbors_v3.get_max_num_nbrs(
            scaling_matrix=scaling_matrix,
            rcut=rcut,
            coords_are_cartesian=coords_are_cartesian)
)

print()
print("Step 2. primitive_cell 中原子的近邻原子情况:")
key_nbr_species, key_nbr_distances, key_nbr_coords = \
            neighbors_v3._get_key_neighs_info(
                            scaling_matrix=scaling_matrix,
                            rcut=rcut,
                            coords_are_cartesian=coords_are_cartesian)
print("\t1.1. The number of atoms in primitive cell:\t", len(neighbors_v3.structure.species))
print("\t1.2. The shape of key_nbr_species:\t", key_nbr_species.shape)
print("\t1.3. The shape of key_nbr_distances:\t", key_nbr_distances.shape)
print("\t1.4. The shape of key_nbr_coords:\t", key_nbr_coords.shape)

print("Running time: {0} s".format(_end-_start))


Step 1. 在截断半径 3.2 内，最大近邻原子数为:	13

Step 2. primitive_cell 中原子的近邻原子情况:
	1.1. The number of atoms in primitive cell:	 12
	1.2. The shape of key_nbr_species:	 (12, 13)
	1.3. The shape of key_nbr_distances:	 (12, 13)
	1.4. The shape of key_nbr_coords:	 (12, 13, 3)
Running time: 0.007814742624759674 s
